
# 🌟 LLMs for Beginners — Hands-on Notebook

This notebook gives you **simple, concrete demos** of how large language models (LLMs) work:
1. **Next-token prediction** (how LLMs generate text)  
2. **Tokenization** (splitting text into tokens)  
3. **Vector embeddings** (turning text into vectors for similarity & search)  

> Designed to run with minimal dependencies. Optional cells use `transformers` and `sentence-transformers` if you have them installed.



## ✅ Setup
This notebook works **out of the box** using only `numpy` and `scikit-learn` (for a basic embeddings demo).  
If you want to try **real LLM tokenizers/embeddings**, install:
```bash
pip install transformers sentence-transformers
```



## 1) 📖 Next-token prediction (the core idea)

LLMs generate text by predicting the **next token** given previous tokens.  
Below is a **toy language model** to illustrate the idea without any external libraries.


In [ ]:

import numpy as np
rng = np.random.default_rng(42)

# Tiny vocabulary + conditional next-token probabilities (toy)
vocab = ["I", "love", "like", "cats", "dogs", ".", "and"]
probs = {
    "START": {"I": 1.0},
    "I": {"love": 0.6, "like": 0.4},
    "love": {"cats": 0.5, "dogs": 0.5},
    "like": {"cats": 0.7, "dogs": 0.3},
    "cats": {"and": 0.3, ".": 0.7},
    "dogs": {"and": 0.3, ".": 0.7},
    "and": {"cats": 0.5, "dogs": 0.5},
    ".": {}
}

def sample_next(prev):
    dist = probs.get(prev, {})
    if not dist:
        return "."
    tokens = list(dist.keys())
    p = np.array([dist[t] for t in tokens], dtype=float)
    p = p / p.sum()
    return rng.choice(tokens, p=p)

def generate(max_tokens=8):
    seq = ["I"]  # start tokenized as "I" from our toy START
    while len(seq) < max_tokens:
        nxt = sample_next(seq[-1])
        seq.append(nxt)
        if nxt == ".":
            break
    return " ".join(seq)

for _ in range(3):
    print(generate())



### (Optional) Real model generation with 🤗 Transformers
If you have `transformers` installed, try a small text generator.


In [ ]:

try:
    from transformers import pipeline
    gen = pipeline("text-generation", model="gpt2")
    out = gen("Rwanda is known for", max_new_tokens=20, do_sample=True, top_p=0.9)
    print(out[0]["generated_text"])
except Exception as e:
    print("Skipping transformers demo (install transformers, or model download failed). Error:", e)



## 2) ✂️ Tokenization (turning text → tokens)

Tokenizers split text into **tokens** (words, subwords, or characters).  
Modern LLMs use **subword tokenization** (BPE, WordPiece) to handle rare words efficiently.


In [ ]:

text = "Transformers make language tasks easier in Rwanda."
print("Raw text:", text)

# Simple whitespace tokenizer (concept demo)
tokens_ws = text.split()
print("Whitespace tokens:", tokens_ws)
print("Number of tokens (whitespace):", len(tokens_ws))


In [ ]:

# (Optional) Use a real tokenizer with 🤗 Transformers if available
try:
    from transformers import AutoTokenizer
    tok = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    ids = tok.encode(text, add_special_tokens=True)
    print("Real tokenizer IDs:", ids)
    print("Decoded back:", tok.decode(ids))
    print("Number of tokens (subword):", len(ids))
except Exception as e:
    print("Skipping real tokenizer demo. Error:", e)



## 3) 🔢 Vector Embeddings (text → vectors)

Embeddings map text to **vectors** so we can compute **similarity**.  
We’ll show two paths:
1. **TF-IDF** (classic, sparse vectors) – no extra installs.
2. **Sentence embeddings** (dense, semantic vectors) – needs `sentence-transformers`.


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

docs = [
    "Kigali hosts data science workshops.",
    "We teach beginners about machine learning.",
    "Rwanda has fast-growing tech communities.",
    "Neural networks power modern AI."
]
query = "Intro workshop on machine learning in Kigali"

vec = TfidfVectorizer()
X = vec.fit_transform(docs + [query])
sims = cosine_similarity(X[-1], X[:-1]).ravel()

for doc, s in sorted(zip(docs, sims), key=lambda x: -x[1]):
    print(f"{s:.3f} :: {doc}")


In [ ]:

# (Optional) Semantic embeddings with sentence-transformers
try:
    from sentence_transformers import SentenceTransformer
    import numpy as np

    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    emb = model.encode(docs + [query], normalize_embeddings=True)
    sims = emb[-1] @ emb[:-1].T  # cosine since normalized

    for doc, s in sorted(zip(docs, sims), key=lambda x: -x[1]):
        print(f"{s:.3f} :: {doc}")
except Exception as e:
    print("Skipping sentence-transformers demo. Error:", e)



### 🧪 Mini Exercise — Build a tiny vector search
- Replace the `docs` list with 6–10 short FAQs from your domain.  
- Keep `query` as a user question.  
- Run the **TF-IDF** cell to see which doc matches best.  
- (Optional) Try **sentence-transformers** for better semantic matching.



## 📈 Visual intuition (optional)

If you have `matplotlib` and `sklearn` installed, you can project embeddings to 2D with PCA to **see** how similar texts cluster.


In [ ]:

try:
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.decomposition import PCA

    # Use TF-IDF vectors for a quick plot
    X = vec.fit_transform(docs).toarray()
    pts = PCA(n_components=2).fit_transform(X)
    plt.figure()
    plt.scatter(pts[:,0], pts[:,1])
    for i, d in enumerate(docs):
        plt.annotate(str(i+1), (pts[i,0], pts[i,1]))
    plt.title("Docs in 2D (TF-IDF + PCA)")
    plt.show()
except Exception as e:
    print("Skipping visualization. Error:", e)



## ✅ Wrap-up
- **LLMs** predict the **next token**, one step at a time.  
- **Tokenization** turns text into model-friendly token IDs (often subwords).  
- **Embeddings** map text to vectors so we can compute **similarity** and do **search**.

**Where to go next**
- Try Hugging Face pipelines for summarization, Q&A, and text generation.  
- Swap in your **own documents** to build a tiny **semantic search** demo.
